In [4]:
# Instale as bibliotecas necessárias
!pip install gradio pandas nltk

In [5]:
import gradio as gr
import openai

# Função para avaliação de desempenho do funcionário
def avaliar_funcionario(emp_id):
    # Simulação de avaliação com base no ID do funcionário
    if emp_id == "101":
        return "Manter"
    elif emp_id == "102":
        return "Treinar"
    elif emp_id == "103":
        return "Reunião"
    else:
        return "Demitir"

# Função de IA para dar opinião sobre o desempenho
def opiniao_ia(desempenho_resultado):
    if desempenho_resultado == "Manter":
        return "🟢 O funcionário está com bom desempenho. Continue com a motivação e ofereça mais desafios para crescimento."
    elif desempenho_resultado == "Treinar":
        return "🟡 O funcionário tem desempenho satisfatório, mas pode melhorar com treinamentos. Considere direcionar para cursos ou mentorias."
    elif desempenho_resultado == "Reunião":
        return "🔴 O desempenho do funcionário precisa de uma reunião para avaliar a situação. Pode ser necessário alinhar expectativas e objetivos."
    elif desempenho_resultado == "Demitir":
        return "⚠️ O desempenho do funcionário está abaixo do esperado. É importante considerar uma conversa para melhorar ou tomar outras ações."
    else:
        return "❓ Análise inconclusiva, é necessário mais dados."

# Função para analisar o sentimento da frase
def analisar_sentimento(frase):
    # Simulando uma análise de sentimento
    if "feliz" in frase or "bom" in frase:
        return "Felicidade"
    elif "triste" in frase or "mal" in frase:
        return "Tristeza"
    else:
        return "Neutro"

# Função para IA gerar opinião sobre o sentimento
def opiniao_sentimento(sentimento):
    if "felicidade" in sentimento.lower():
        return "😊 Parece que o funcionário está bem. Incentive mais interações positivas!"
    elif "tristeza" in sentimento.lower():
        return "😞 O funcionário parece estar enfrentando desafios. Talvez seja interessante uma conversa de apoio e motivação."
    else:
        return "😐 O sentimento é neutro. É importante continuar monitorando para um melhor entendimento."

# Função para adicionar candidatos à lista
candidatos = []
def adicionar_candidato(nome, experiencia, habilidades, formacao):
    candidato = {
        "nome": nome,
        "experiencia": int(experiencia),
        "habilidades": habilidades.split(","),
        "formacao": formacao
    }
    candidatos.append(candidato)
    return f"Candidato {nome} adicionado com sucesso!"

# Função para filtrar os melhores candidatos
def filtrar_candidatos(exp_min, habilidades_req, formacao_req):
    exp_min = int(exp_min)
    habilidades_req = set(habilidades_req.split(","))
    candidatos_filtrados = []

    for candidato in candidatos:
        if candidato["experiencia"] >= exp_min:
            habilidades_candidato = set(candidato["habilidades"])
            if habilidades_candidato.intersection(habilidades_req) and candidato["formacao"] == formacao_req:
                candidatos_filtrados.append(candidato["nome"])

    return "\n".join(candidatos_filtrados) if candidatos_filtrados else "Nenhum candidato encontrado."

# Função para registrar a resposta do funcionário
mensagens_colaboradores = []
def registrar_resposta_funcionario(nome, frase):
    sentimento = analisar_sentimento(frase)
    mensagens_colaboradores.append((nome, frase, sentimento))
    return f"Mensagem registrada! Sentimento: {sentimento}"

# Função para exibir as respostas dos funcionários para o RH
def exibir_respostas_rh():
    return "\n".join([f"{nome}: {mensagem} - Sentimento: {sentimento}" for nome, mensagem, sentimento in mensagens_colaboradores])

# Função para responder dúvidas gerais (ChatGPT-like)
def responder_duvida(pergunta):
    # Aqui podemos usar a API do OpenAI para responder dúvidas gerais
    openai.api_key = 'YOUR_OPENAI_API_KEY'  # Substitua pela sua chave da OpenAI

    response = openai.Completion.create(
        engine="text-davinci-003",  # Modelo GPT-3
        prompt=pergunta,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Criando a interface gráfica com Gradio
with gr.Blocks(css=".gradio-container {font-family: 'Arial', sans-serif; background-color: #f9f9f9;}") as app:
    # Cabeçalho
    with gr.Row():
        gr.Markdown(
            "<h1 style='text-align: center; color: #4CAF50;'>RHAI - A sua I.A de RH</h1>"
        )

    with gr.Tabs():
        # Aba Desempenho do Funcionário
        with gr.Tab("📊 Desempenho do Funcionário"):
            gr.Markdown("### Avaliação de Desempenho\nInsira o **ID do Funcionário** para avaliar seu status.")
            emp_id_input = gr.Textbox(label="ID do Funcionário", placeholder="Ex.: 101")
            desempenho_output = gr.Textbox(label="Resultado da Avaliação", interactive=False)
            opiniao_output = gr.Textbox(label="Opinião da IA", interactive=False)
            avaliar_btn = gr.Button("📈 Avaliar", variant="primary")
            avaliar_btn.click(avaliar_funcionario, inputs=emp_id_input, outputs=desempenho_output)
            avaliar_btn.click(opiniao_ia, inputs=desempenho_output, outputs=opiniao_output)

        # Aba Seleção de Candidatos
        with gr.Tab("👥 Seleção de Candidatos"):
            with gr.Row():
                nome = gr.Textbox(label="Nome do Candidato")
                experiencia = gr.Textbox(label="Anos de Experiência")
                habilidades = gr.Textbox(label="Habilidades (separadas por vírgula)")
                formacao = gr.Textbox(label="Formação")
                adicionar_btn = gr.Button("➕ Adicionar Candidato")
                adicionar_output = gr.Textbox(label="Status", interactive=False)
                adicionar_btn.click(adicionar_candidato, inputs=[nome, experiencia, habilidades, formacao], outputs=adicionar_output)
            with gr.Row():
                exp_min = gr.Textbox(label="Experiência Mínima")
                habilidades_req = gr.Textbox(label="Habilidades Necessárias")
                formacao_req = gr.Textbox(label="Formação Necessária")
                filtrar_btn = gr.Button("🔍 Filtrar Candidatos")
                filtro_output = gr.Textbox(label="Melhores Candidatos", lines=5, interactive=False)
                filtrar_btn.click(filtrar_candidatos, inputs=[exp_min, habilidades_req, formacao_req], outputs=filtro_output)

        # Aba Análise de Sentimento
        with gr.Tab("😊 Análise de Sentimento"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Registro de Sentimentos dos Funcionários")
                    nome_funcionario = gr.Textbox(label="Seu Nome", placeholder="Ex.: João Silva")
                    frase_input = gr.Textbox(label="Descreva seu dia", placeholder="Escreva algo aqui...")
                    registrar_btn = gr.Button("✅ Registrar Resposta")
                    registrar_output = gr.Textbox(label="Status", interactive=False)
                    registrar_btn.click(registrar_resposta_funcionario, inputs=[nome_funcionario, frase_input], outputs=registrar_output)

                with gr.Column():
                    gr.Markdown("### Visualização pelo RH")
                    rh_output = gr.Textbox(label="Respostas dos Funcionários", interactive=False, lines=10)
                    exibir_btn = gr.Button("🔍 Exibir Respostas", variant="secondary")
                    exibir_btn.click(exibir_respostas_rh, inputs=None, outputs=rh_output)
                    gr.Markdown("### Opinião da IA sobre Sentimentos")
                    sentimento_output = gr.Textbox(label="Sentimento", interactive=False)
                    opiniao_sentimento_output = gr.Textbox(label="Opinião da IA sobre o Sentimento", interactive=False)
                    exibir_btn.click(lambda x: analisar_sentimento(x), inputs=rh_output, outputs=sentimento_output)
                    exibir_btn.click(opiniao_sentimento, inputs=sentimento_output, outputs=opiniao_sentimento_output)

        # Aba Dúvidas Gerais (ChatGPT)
        with gr.Tab("🤖 Dúvidas Gerais"):
            gr.Markdown("### Pergunte à IA sobre qualquer coisa!")
            pergunta_input = gr.Textbox(label="Digite sua pergunta", placeholder="Faça sua pergunta aqui...")
            resposta_output = gr.Textbox(label="Resposta da IA", interactive=False)
            perguntar_btn = gr.Button("💬 Perguntar")
            perguntar_btn.click(responder_duvida, inputs=pergunta_input, outputs=resposta_output)

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52c8a83e0100fc5a4a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
